# BOM

```
Bicycle
	Wheel
		Spokes
		Hub
		Rim
			Color
		Tire
		Valve
	Front Set
		Handlebar grip
		Front brake
		Fork
			Color
		Shifter
	Saddle Area
		Saddle
		Seat post
	Transmission
		Chain
		Chain Rings
		Rear brakes
		Cog set
		Rear Derailluer
		Front Derailleur
		Pedal
		Crank Arm
	Frame
		Frame
		Stem
		Head
			Color
```


## Relationships



### Supplier
 - Supplier supplies Part
 - Supplier supplies Assembly


### Person
 - Person assembles Part
 - Person assembles Assembly


### Assembly
 - Assembly is sourced from Supplier
 - Assembly belongs to Product
 - Assembly has Part
 - Assembly has Assembly
 - Assembly is assembled by Person
 - Assembly is specified by Specification
 - Assembly is instructed by Instruction


### Part
 - Part is assembled by Person
 - Part is sourced from Supplier
 - Part belongs to Product
 - Part belongs to Assembly
 - Part is specified by Specification
 - Part is instructed by Instruction


### Product
 - Product has Assembly
 - Product has Part 
 - Product is specified by Specification
 - Product is instructed by Instruction


### Specification
 - Specification specifies Part
 - Specification specifies Assembly
 - Specification specifies Product


### Instruction
 - Instruction instructs Part
 - Instruction instructs Assembly
 - Instruction instructs Product


## Queries

```
// Delete all
MATCH (n) DETACH DELETE (n);


// Bicycle
CREATE CONSTRAINT b_sku_unique IF NOT EXISTS FOR (n:Bicycle) REQUIRE n.sku IS UNIQUE;
CREATE (b:Bicycle {sku: 'ss01', name: 'SuperSport', retail_price: 29900});


// SubAssembly
CREATE CONSTRAINT s_sku_unique IF NOT EXISTS FOR (n:SubAssembly) REQUIRE n.sku IS UNIQUE;
CREATE (sa:SubAssembly {sku: 'whl01', name: 'Wheel SubAssembly'});
CREATE (sa:SubAssembly {sku: 'fs01', name: 'Front Set'});
CREATE (sa:SubAssembly {sku: 'sa01', name: 'Saddle Area'});

// Parts
CREATE CONSTRAINT p_sku_unique IF NOT EXISTS FOR (n:Part) REQUIRE n.sku IS UNIQUE;
CREATE (p:Part {sku: 'sp01', name: 'Spoke', cost: 15});
CREATE (p:Part {sku: 'hb01', name: 'Hub', cost: 89});
CREATE (p:Part {sku: 'rm01', name: 'Rim', cost: 1158});
CREATE (p:Part {sku: 'ty01', name: 'Tire', cost: 568});
CREATE (p:Part {sku: 'vlv01', name: 'Valve', cost: 451});
CREATE (p:Part {sku: 'hdg01', name: 'Handlebar grip', cost: 745});
CREATE (p:Part {sku: 'fb01', name: 'Front brake', cost: 451});
CREATE (p:Part {sku: 's01', name: 'Saddle', cost: 111});
CREATE (p:Part {sku: 'sp02', name: 'Seat post', cost: 222});
CREATE (p:Part {sku: 'fd01', name: 'Front Derailleur', cost: 451});
CREATE (p:Part {sku: 'ch01', name: 'Chain', cost: 234});
CREATE (p:Part {sku: 'cr01', name: 'Chain Rings', cost: 456});
CREATE (p:Part {sku: 'fr01', name: 'Frame', cost: 444});
CREATE (p:Part {sku: 'rb01', name: 'Rear brakes', cost: 261});
CREATE (p:Part {sku: 'cs01', name: 'Cog set', cost: 1256});
CREATE (p:Part {sku: 'rd01', name: 'Rear Derailluer', cost: 6432});
CREATE (p:Part {sku: 'pd01', name: 'Pedal', cost: 311});


// Bike has Sub-assemblies
MATCH (b:Bicycle),(sa:SubAssembly) WHERE b.sku = 'ss01' AND sa.sku = 'whl01' CREATE (b)-[:HAS {qty: 2, order: 1}]->(sa);
MATCH (b:Bicycle),(sa:SubAssembly) WHERE b.sku = 'ss01' AND sa.sku = 'fs01' CREATE (b)-[:HAS {qty: 1, order: 2}]->(sa);
MATCH (b:Bicycle),(sa:SubAssembly) WHERE b.sku = 'ss01' AND sa.sku = 'sa01' CREATE (b)-[:HAS {qty: 1, order: 3}]->(sa);


// Sub-assemblies makes up bike
MATCH (b:Bicycle),(sa:SubAssembly) WHERE b.sku = 'ss01' AND sa.sku = 'whl01' CREATE (sa)-[:MAKES_UP {qty: 2, order: 1}]->(b);
MATCH (b:Bicycle),(sa:SubAssembly) WHERE b.sku = 'ss01' AND sa.sku = 'fs01' CREATE (sa)-[:MAKES_UP {qty: 1, order: 2}]->(b);
MATCH (b:Bicycle),(sa:SubAssembly) WHERE b.sku = 'ss01' AND sa.sku = 'sa01' CREATE (sa)-[:MAKES_UP {qty: 1, order: 3}]->(b);


// Parts makes up bike
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'hb01' CREATE (p)-[:MAKES_UP {qty: 2, order: 1}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'sp01' CREATE (p)-[:MAKES_UP {qty: 42, order: 2}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'rm01' CREATE (p)-[:MAKES_UP {qty: 2, order: 3}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'vlv01' CREATE (p)-[:MAKES_UP {qty: 2, order: 4}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'ty01' CREATE (p)-[:MAKES_UP {qty: 2, order: 5}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'hdg01' CREATE (p)-[:MAKES_UP {qty: 2, order: 1}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'fb01' CREATE (p)-[:MAKES_UP {qty: 1, order: 2}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 's01' CREATE (p)-[:MAKES_UP {qty: 1, order: 1}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'sp02' CREATE (p)-[:MAKES_UP {qty: 1, order: 2}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'fd01' CREATE (p)-[:MAKES_UP {qty: 1, order: 4}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'ch01' CREATE (p)-[:MAKES_UP {qty: 1, order: 5}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'cr01' CREATE (p)-[:MAKES_UP {qty: 1, order: 6}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'fr01' CREATE (p)-[:MAKES_UP {qty: 1, order: 7}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'rb01' CREATE (p)-[:MAKES_UP {qty: 1, order: 8}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'cs01' CREATE (p)-[:MAKES_UP {qty: 1, order: 9}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'rd01' CREATE (p)-[:MAKES_UP {qty: 1, order: 10}]->(b);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'pd01' CREATE (p)-[:MAKES_UP {qty: 2, order: 11}]->(b);

// Bike has Parts
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'hb01' CREATE (b)-[:HAS {qty: 2, order: 1}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'sp01' CREATE (b)-[:HAS {qty: 42, order: 2}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'rm01' CREATE (b)-[:HAS {qty: 2, order: 3}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'vlv01' CREATE (b)-[:HAS {qty: 2, order: 4}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'ty01' CREATE (b)-[:HAS {qty: 2, order: 5}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'hdg01' CREATE (b)-[:HAS {qty: 2, order: 1}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'fb01' CREATE (b)-[:HAS {qty: 1, order: 2}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 's01' CREATE (b)-[:HAS {qty: 11, order: 1}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'sp02' CREATE (b)-[:HAS {qty: 1, order: 2}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'fd01' CREATE (b)-[:HAS {qty: 1, order: 4}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'ch01' CREATE (b)-[:HAS {qty: 1, order: 5}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'cr01' CREATE (b)-[:HAS {qty: 1, order: 6}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'fr01' CREATE (b)-[:HAS {qty: 1, order: 7}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'rb01' CREATE (b)-[:HAS {qty: 1, order: 8}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'cs01' CREATE (b)-[:HAS {qty: 1, order: 9}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'rd01' CREATE (b)-[:HAS {qty: 1, order: 10}]->(p);
MATCH (b:Bicycle),(p:Part) WHERE b.sku = 'ss01' AND p.sku = 'pd01' CREATE (b)-[:HAS {qty: 2, order: 11}]->(p);


// Parts makes up Sub-assemblies
// Wheel
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'whl01' AND p.sku = 'hb01' CREATE (p)-[:MAKES_UP {qty: 1, order: 1}]->(sa);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'whl01' AND p.sku = 'sp01' CREATE (p)-[:MAKES_UP {qty: 21, order: 2}]->(sa);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'whl01' AND p.sku = 'rm01' CREATE (p)-[:MAKES_UP {qty: 1, order: 3}]->(sa);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'whl01' AND p.sku = 'vlv01' CREATE (p)-[:MAKES_UP {qty: 1, order: 4}]->(sa);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'whl01' AND p.sku = 'ty01' CREATE (p)-[:MAKES_UP {qty: 1, order: 5}]->(sa);
// Front Set
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'sa01' AND p.sku = 'hdg01' CREATE (p)-[:MAKES_UP {qty: 2, order: 1}]->(sa);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'sa01' AND p.sku = 'fb01' CREATE (p)-[:MAKES_UP {qty: 1, order: 2}]->(sa);
// Saddle Area
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'fs01' AND p.sku = 's01' CREATE (p)-[:MAKES_UP {qty: 11, order: 1}]->(sa);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'fs01' AND p.sku = 'sp02' CREATE (p)-[:MAKES_UP {qty: 1, order: 2}]->(sa);


// Sub-assembly has parts
// Wheel
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'whl01' AND p.sku = 'hb01' CREATE (sa)-[:HAS {qty: 1, order: 1}]->(p);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'whl01' AND p.sku = 'sp01' CREATE (sa)-[:HAS {qty: 21, order: 2}]->(p);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'whl01' AND p.sku = 'rm01' CREATE (sa)-[:HAS {qty: 1, order: 3}]->(p);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'whl01' AND p.sku = 'vlv01' CREATE (sa)-[:HAS {qty: 1, order: 4}]->(p);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'whl01' AND p.sku = 'ty01' CREATE (sa)-[:HAS {qty: 1, order: 5}]->(p);
// Front Set
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'sa01' AND p.sku = 'hdg01' CREATE (sa)-[:HAS {qty: 2, order: 1}]->(p);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'sa01' AND p.sku = 'fb01' CREATE (sa)-[:HAS {qty: 1, order: 2}]->(p);
// Saddle Area
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'fs01' AND p.sku = 's01' CREATE (sa)-[:HAS {qty: 1, order: 1}]->(p);
MATCH (sa:SubAssembly), (p:Part) WHERE sa.sku = 'fs01' AND p.sku = 'sp02' CREATE (sa)-[:HAS {qty: 1, order: 2}]->(p);

```


In [1]:
!pip install lorem-text

In [2]:
from lorem_text import lorem

In [3]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://89698250.databases.neo4j.io"
AUTH = ("neo4j", "nN4v9Y33RjQtEPzqgNAUUbKQj6Os9Hbs_AuGSY7s1DQ")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [108]:
# Clear the DB
records, summary, keys = driver.execute_query(
    "MATCH (n) DETACH DELETE (n);",
    database_="neo4j",
)

In [109]:
queries = []

# Set unique constraints
queries.append('CREATE CONSTRAINT s_name_unique IF NOT EXISTS FOR (n:Supplier) REQUIRE n.name IS UNIQUE;')
queries.append('CREATE CONSTRAINT s_sku_unique IF NOT EXISTS FOR (n:Assembly) REQUIRE n.sku IS UNIQUE;')
queries.append('CREATE CONSTRAINT b_sku_unique IF NOT EXISTS FOR (n:Product) REQUIRE n.sku IS UNIQUE;')
queries.append('CREATE CONSTRAINT p_sku_unique IF NOT EXISTS FOR (n:Part) REQUIRE n.sku IS UNIQUE;')
queries.append('CREATE CONSTRAINT d_sku_version_unique IF NOT EXISTS FOR (n:Description) REQUIRE (n.sku, n.version) IS UNIQUE;')
queries.append('CREATE CONSTRAINT s_sku_version_unique IF NOT EXISTS FOR (n:Specification) REQUIRE (n.sku, n.version) IS UNIQUE;')

# Execute Queries
for query in queries:
    records, summary, keys = driver.execute_query(
    query,
    database_="neo4j",
)


In [110]:
# Create Suppliers
queries = []

suppliers = [
    {'code': 'FR001', 'name': 'XYZ Co (self)', 'country': "FR"},
    {'code': 'FR002', 'name': 'Supplier #1', 'country': "FR"},
    {'code': 'CN001', 'name': 'Supplier #2', 'country': "CN"},
    {'code': 'US001', 'name': 'Supplier #3', 'country': "US"},
    {'code': 'UK001', 'name': 'Supplier #4', 'country': "UK"},
]

# Create Suppliers
for supplier in suppliers:
    code = supplier['code']
    name = supplier['name']
    country = supplier['country']
    
    query = f"CREATE (:Supplier {{code: '{code}', name: '{name}', country: '{country}'}})"
    queries.append(query)

# Execute Queries
for query in queries:
    records, summary, keys = driver.execute_query(
    query,
    database_="neo4j",
)


In [111]:
# Create Parts and relationships
queries = []

parts = [
    {'sku': 'sp01', 'name': 'Spoke', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 100, 'selected': True},
            {'code': 'FR002', 'cost': 110},
            {'code': 'US001', 'cost': 120},
    ]},
    {'sku': 'hb01', 'name': 'Hub', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 60},
            {'code': 'US001', 'cost': 89},
    ]},
    {'sku': 'rm01', 'name': 'Rim', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 600},
            {'code': 'US001', 'cost': 510},
    ]},
    {'sku': 'ty01', 'name': 'Tire', 'uom': 'pc', 'suppliers': [
            {'code': 'FR002', 'cost': 568},
            {'code': 'CN001', 'cost': 320},
            {'code': 'US001', 'cost': 700},
    ]},
    {'sku': 'vlv01', 'name': 'Valve', 'uom': 'pc', 'suppliers': [
            {'code': 'FR002', 'cost': 42},
            {'code': 'CN001', 'cost': 25},
            {'code': 'US001', 'cost': 32},
    ]},
    {'sku': 'hbr01', 'name': 'Handlebar', 'uom': 'pc', 'suppliers': [
            {'code': 'FR002', 'cost': 234},
    ]},
    {'sku': 'hdg01', 'name': 'Handlebar grip', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 121},
            {'code': 'FR002', 'cost': 234},
            {'code': 'US001', 'cost': 210},
    ]},
    {'sku': 'fb01', 'name': 'Front brake', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 451},
            {'code': 'US001', 'cost': 612},
    ]},
    {'sku': 'fo01', 'name': 'Fork', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 213},
            {'code': 'FR001', 'cost': 250},
            {'code': 'FR002', 'cost': 300},
            {'code': 'US001', 'cost': 320},
    ]},
    {'sku': 'sh01', 'name': 'Shifter', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 221},
            {'code': 'US001', 'cost': 240},
    ]},
    {'sku': 's01', 'name': 'Saddle', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 111},
            {'code': 'UK001', 'cost': 125},
    ]},
    {'sku': 'sp02', 'name': 'Seat post', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 222},
            {'code': 'US001', 'cost': 300},
    ]},
    {'sku': 'fd01', 'name': 'Front Derailleur', 'uom': 'pc', 'suppliers': [
            {'code': 'US001', 'cost': 451},
    ]},
    {'sku': 'ch01', 'name': 'Chain', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 234},
    ]},
    {'sku': 'cr01', 'name': 'Chain Rings', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 60},
            {'code': 'FR002', 'cost': 60},
            {'code': 'US001', 'cost': 89},
    ]},
    {'sku': 'fr01', 'name': 'Frame', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 212},
            {'code': 'FR001', 'cost': 250},
            {'code': 'FR002', 'cost': 300},
            {'code': 'US001', 'cost': 320},
    ]},
    {'sku': 'rb01', 'name': 'Rear brakes', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 60},
            {'code': 'US001', 'cost': 89},
    ]},
    {'sku': 'cs01', 'name': 'Cog set', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 261},
            {'code': 'US001', 'cost': 211},
    ]},
    {'sku': 'rd01', 'name': 'Rear Derailluer', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 340},
            {'code': 'US001', 'cost': 432},
    ]},
    {'sku': 'pd01', 'name': 'Pedal', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 60},
            {'code': 'FR002', 'cost': 80},
            {'code': 'US001', 'cost': 62},
    ]},
    {'sku': 'ca01', 'name': 'Crank Arm', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 60},
            {'code': 'FR001', 'cost': 120},
            {'code': 'US001', 'cost': 89},
    ]},
    {'sku': 'drk_grn', 'name': 'Green', 'uom': 'm2', 'suppliers': [
            {'code': 'FR001', 'cost': 100},
    ]},
    {'sku': 'st01', 'name': 'Stem', 'uom': 'pc', 'suppliers': [
            {'code': 'FR001', 'cost': 400},
    ]},
    {'sku': 'hd01', 'name': 'Head', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 500},
            {'code': 'US001', 'cost': 550},
    ]},
]

for part in parts:
    sku = part['sku']
    name = part['name']
    uom = part['uom']
    
    query = f"CREATE (:Part {{sku: '{sku}', name: '{name}', uom: '{uom}'}})"
    queries.append(query)
    
    # Part Description
    query = f"CREATE (:Description {{sku: '{sku}', body: '{lorem.sentence()}', version: 'latest'}})"
    queries.append(query)
    query = f"MATCH (p:Part {{sku: '{sku}'}}), (d:Description {{sku: '{sku}'}}) CREATE (p)-[:IS_DESCRIBED_BY]->(d)"
    queries.append(query)
#     query = f"MATCH (p:Part {{sku: '{sku}'}}), (d:Description {{sku: '{sku}'}}) CREATE (d)-[:DESCRIBES]->(p)"
#     queries.append(query)

    # Part Specification
    query = f"CREATE (:Specification {{sku: '{sku}', body: '{lorem.sentence()}', version: 'latest'}})"
    queries.append(query)
    query = f"MATCH (p:Part {{sku: '{sku}'}}), (d:Specification {{sku: '{sku}'}}) CREATE (p)-[:IS_SPECIFIED_BY]->(d)"
    queries.append(query)
#     query = f"MATCH (p:Part {{sku: '{sku}'}}), (d:Specification {{sku: '{sku}'}}) CREATE (d)-[:SPECIFIES]->(p)"
#     queries.append(query)

    counter = 0
    for supplier in part['suppliers']:
        supplier_code = supplier['code']
        supplier_cost = supplier['cost']
        
	# Supplier
        part_supplier_query = f"MATCH (p:Part {{sku: '{sku}'}}), (s:Supplier {{code: '{supplier_code}'}}) CREATE (p)-[:IS_SOURCED_FROM {{cost: {supplier_cost}}}]->(s)"
        queries.append(part_supplier_query)
        # supplier_part_query = f"MATCH (p:Part {{sku: '{sku}'}}), (s:Supplier {{code: '{supplier_code}'}}) CREATE (s)-[:SUPPLIES {{cost: {supplier_cost}}}]->(p)"
        # queries.append(supplier_part_query)

	# Selected Supplier / Winning Supplier / Contracted Supplier
        if counter == 0:
            selected_supplier_query = f"MATCH (p:Part {{sku: '{sku}'}}), (s:Supplier {{code: '{supplier_code}'}}) CREATE (p)-[:SELECTED {{cost: {supplier_cost}}}]->(s)"
            queries.append(selected_supplier_query)

        counter +=1
        


# Print the queries
# for query in queries:
#     print(query)
    
# Execute Queries
for query in queries:
    records, summary, keys = driver.execute_query(
    query,
    database_="neo4j",
)



In [112]:
# Create Product(s) and relationships
queries = []

products = [
    {
        "sku": "ss01",
        "name": "SuperSport",
        "retail_price": 29999,
        "assemblies": [
            {'sku': 'co01', 'name': 'Color', 'parts': [
                {'sku':'drk_grn','qty':2, 'order': 1},
                ]},
            {'sku': 'whl01', 'name': 'Wheel Assembly', 'parts': [
                {'sku':'sp01','qty':21, 'order': 2},
                {'sku':'hb01','qty':1, 'order': 1},
                {'sku':'rm01','qty':1, 'order': 3},
                {'sku':'ty01','qty':1, 'order': 5},
                {'sku':'vlv01','qty':1, 'order': 4},
                ], 'assemblies': [
                    {'sku': 'co01'}
                ]},
            {'sku': 'fs01', 'name': 'Front Set', 'parts': [
                {'sku':'hbr01','qty':1, 'order': 1},
                {'sku':'hdg01','qty':2, 'order': 2},
                {'sku':'fb01','qty':1, 'order': 3},
                {'sku':'fo01','qty':1, 'order': 5},
                {'sku':'sh01','qty':1, 'order': 4},
                ], 'assemblies': [
                    {'sku': 'co01'}
                ]},
            {'sku': 'sa01', 'name': 'Saddle Area', 'parts': [
                {'sku':'s01','qty':1, 'order': 2},
                {'sku':'sp02','qty':1, 'order': 1},
                ]},
            {'sku': 'tr01', 'name': 'Transmission', 'parts': [
                {'sku':'fd01','qty':21, 'order': 2},
                {'sku':'ch01','qty':1, 'order': 1},
                {'sku':'cr01','qty':1, 'order': 3},
                {'sku':'rb01','qty':1, 'order': 4},
                {'sku':'cs01','qty':1, 'order': 5},
                {'sku':'rd01','qty':1, 'order': 6},
                {'sku':'rca01','qty':2, 'order': 7},
                {'sku':'pd01','qty':2, 'order': 8},
                ]},
            {'sku': 'fr01', 'name': 'Frame', 'parts': [
                {'sku':'fr01','qty':1, 'order': 1},
                {'sku':'st01','qty':1, 'order': 2},
                {'sku':'hd01','qty':1, 'order': 3},
                ], 'assemblies': [
                    {'sku': 'co01'}
                ]},
        ],
    }
]

for product in products:
    sku = product['sku']
    name = product['name']
    retail_price = product['retail_price']

    # Create Product
    product_query = f"CREATE (:Product {{sku: '{sku}', name: '{name}', retail_price: {retail_price}}})"
    queries.append(product_query)
    
    # Product Description
    query = f"CREATE (:Description {{sku: '{sku}', body: '{lorem.sentence()}', version: 'latest'}})"
    queries.append(query)
    query = f"MATCH (p:Product {{sku: '{sku}'}}), (d:Description {{sku: '{sku}'}}) CREATE (p)-[:IS_DESCRIBED_BY]->(d)"
    queries.append(query)
    # query = f"MATCH (p:Product {{sku: '{sku}'}}), (d:Description {{sku: '{sku}'}}) CREATE (d)-[:DESCRIBES]->(p)"
    # queries.append(query)

    # Product Specification
    query = f"CREATE (:Specification {{sku: '{sku}', body: '{lorem.sentence()}', version: 'latest'}})"
    queries.append(query)
    query = f"MATCH (p:Product {{sku: '{sku}'}}), (d:Specification {{sku: '{sku}'}}) CREATE (p)-[:IS_SPECIFIED_BY]->(d)"
    queries.append(query)
    # query = f"MATCH (p:Product {{sku: '{sku}'}}), (d:Specification {{sku: '{sku}'}}) CREATE (d)-[:SPECIFIES]->(p)"
    # queries.append(query)
    
    for assembly in product['assemblies']:
        assembly_sku = assembly['sku']
        assembly_name = assembly['name']
        
        assembly_query = f"CREATE (:Assembly {{sku: '{assembly_sku}', name: '{assembly_name}'}})"
        queries.append(assembly_query)
        
        # Product Assembly
        # assembly_part_query = f"MATCH (a:Assembly {{sku: '{assembly_sku}'}}), (p:Product {{sku: '{sku}'}}) CREATE (a)-[:BELONGS_TO {{qty: {qty}, order: {order}}}]->(p)"
        # queries.append(assembly_part_query)
        part_assembly_query = f"MATCH (a:Assembly {{sku: '{assembly_sku}'}}), (p:Product {{sku: '{sku}'}}) CREATE (p)-[:HAS {{qty: {qty}, order: {order}}}]->(a)"
        queries.append(part_assembly_query)
        
        if 'parts' in assembly:
            for part in assembly['parts']:
                part_sku = part['sku']
                qty = part['qty']
                order = part['order']
                
                # Part Assembly
                assembly_part_query = f"MATCH (a:Assembly {{sku: '{assembly_sku}'}}), (p:Part {{sku: '{part_sku}'}}) CREATE (a)-[:HAS {{qty: {qty}, order: {order}}}]->(p)"
                queries.append(assembly_part_query)
                # part_assembly_query = f"MATCH (a:Assembly {{sku: '{assembly_sku}'}}), (p:Part {{sku: '{part_sku}'}}) CREATE (p)-[:BELONGS_TO {{qty: {qty}, order: {order}}}]->(a)"
                # queries.append(part_assembly_query)
                
        if 'assemblies' in assembly:
            for a in assembly['assemblies']:
                a_sku = a['sku']
                        
                assembly_part_query = f"MATCH (a:Assembly {{sku: '{assembly_sku}'}}), (b:Assembly {{sku: '{a_sku}'}}) CREATE (a)-[:HAS]->(b)"
                queries.append(assembly_part_query)



# # Print the queries
# for query in queries:
#     print(query)

# Execute Queries
for query in queries:
    records, summary, keys = driver.execute_query(
    query,
    database_="neo4j",
)




In [113]:
records, summary, keys = driver.execute_query(
    "MATCH p=()-[]->() RETURN p;",
    database_="neo4j",
)

In [ ]:
# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after,
))

# Loop through results and do something with them
for node in records:
    print(node)

In [4]:
with driver.session(database="neo4j") as session:
  graph = session.run("MATCH p=()-[]->() RETURN p;").graph()

graph


In [5]:
def custom_node_color_mapping(index: int, node: dict):
    # print(node)
    if node["properties"]["label"] == "Part":
        return "green"
    elif node["properties"]["label"] == "Product":
        return "red"
    elif node["properties"]["label"] == "Assembly":
        return "orange"
    elif node["properties"]["label"] == "Specification":
        return "blue"
    elif node["properties"]["label"] == "Description":
        return "cyan"
    elif node["properties"]["label"] == "Supplier":
        return "pink"
    
def custom_node_label_mapping(index: int, node: dict):
    # print(node)
    if node["properties"]["label"] == "Part":
        return node["properties"]["sku"] + " - " + node["properties"]["name"]
    elif node["properties"]["label"] == "Product":
        return node["properties"]["sku"] + " - " + node["properties"]["name"]
    elif node["properties"]["label"] == "Assembly":
        return node["properties"]["sku"] + " - " + node["properties"]["name"]
    elif node["properties"]["label"] == "Specification":
        return "v: " + node["properties"]["version"]
    elif node["properties"]["label"] == "Description":
        return "v: " + node["properties"]["version"]
    elif node["properties"]["label"] == "Supplier":
        return node["properties"]["name"] + " - " + node["properties"]["country"]
    

from yfiles_jupyter_graphs import GraphWidget
w = GraphWidget(graph=graph)
w.directed = False
w.set_graph_layout("orthogonal")
w.set_neighborhood(1,[graph.nodes.get(0).element_id])
w.set_sidebar(enabled = False, start_with = "Neighbourhood")
w.set_overview(False)

w.set_node_color_mapping(custom_node_color_mapping)
# w.set_node_color_mapping(lambda index, node : "blue" if node["properties"]["label"] == "Part" else ("orange" if node["properties"]["label"] == "SubAssembly" else "red"))
w.set_edge_color_mapping(lambda index, edge : "orange")
# w.set_node_label_mapping(lambda index, node : node["properties"]["sku"] + " - " + node["properties"]["name"])
w.set_node_label_mapping(custom_node_label_mapping)
# w.set_node_scale_factor_mapping(lambda index,node: node["properties"]["cost"] / 1000 if node["properties"]["label"] == "Part" else 1)

w.show()


GraphWidget(layout=Layout(height='500px', width='100%'))